In [2]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv


_:bool = load_dotenv(find_dotenv())
client:OpenAI = OpenAI()


In [15]:
from openai.types.chat.chat_completion import ChatCompletion
def get_answer():
    response:ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role":"system","content":"you are a developer and you can explain complex logic in simple words"},
            {"role": "user", "content": "what are operator in python?"},
            # {"role": "user", "content": "what is the meaning of life?"}
            
        ]
    )
    display(response)
    return response.choices[0].message.content

get_answer()

ChatCompletion(id='chatcmpl-8cttZh2PFZnqu0WeyxDbzXo7FQqJM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Operators in Python are special symbols that are used to perform operations on variables and values. They can be used to perform arithmetic, comparison, logical, and other operations. For example, the plus symbol (+) is an operator used for addition, the equals symbol (==) is an operator used for comparison, and the and, or, not are operators used for logical operations. These operators help us manipulate and work with data in our Python programs.', role='assistant', function_call=None, tool_calls=None))], created=1704282017, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=89, prompt_tokens=30, total_tokens=119))

'Operators in Python are special symbols that are used to perform operations on variables and values. They can be used to perform arithmetic, comparison, logical, and other operations. For example, the plus symbol (+) is an operator used for addition, the equals symbol (==) is an operator used for comparison, and the and, or, not are operators used for logical operations. These operators help us manipulate and work with data in our Python programs.'

In [20]:
streaming_response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": "say this is a test"},
    ],
    stream=True
)
print(streaming_response)
for text in streaming_response:
    print(text)
    print(text.choices[0].delta.content)

ChatCompletionChunk(id='chatcmpl-8cuHch9kel9EQZVUKRlS8Jn2Q69GQ', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1704283508, model='gpt-3.5-turbo-1106', object='chat.completion.chunk', system_fingerprint='fp_772e8125bb')

ChatCompletionChunk(id='chatcmpl-8cuHch9kel9EQZVUKRlS8Jn2Q69GQ', choices=[Choice(delta=ChoiceDelta(content='This', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1704283508, model='gpt-3.5-turbo-1106', object='chat.completion.chunk', system_fingerprint='fp_772e8125bb')
This
ChatCompletionChunk(id='chatcmpl-8cuHch9kel9EQZVUKRlS8Jn2Q69GQ', choices=[Choice(delta=ChoiceDelta(content=' is', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1704283508, model='gpt-3.5-turbo-1106', object='chat.completion.chunk', system_fingerprint='fp_772e8125bb')
 is
ChatComple

In [24]:
import json
json_mode = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type":"json_object"},
    messages=[
        {"role":"system","content":"you are a helpful assistant designed to output JSON"},
        {"role": "user", "content": "name the 5 most common programming languages"},          
    ],  
)
print(json_mode.choices[0].message.content)
print(type(json_mode.choices[0].message.content))


{
  "languages": [
    "JavaScript",
    "Python",
    "Java",
    "C#",
    "C++"
  ]
}
<class 'str'>


In [27]:
languages_dict:dict[str,list[str]] = json.loads(json_mode.choices[0].message.content)
for languauge in languages_dict["languages"]:
    print(languauge)

JavaScript
Python
Java
C#
C++


# Function Calling


In [72]:
import json
def get_current_weather(location:str, weather:str = "rainy"):
    if "pakistan" == location.lower():
        return json.dumps({"location":"pakistan","weather":"sunny"})
    elif "india" == location.lower():
        return json.dumps({"location":"india","weather":"windy"})
    else:
        return json.dumps({"location":location,"weather":"unknown"})
    
def get_movie_recommendation(genre:str,rating:float,year:int):
    return json.dumps({"genre":genre,"rating":rating,"year":year})


In [75]:
def my_first_function_call(main_request:str):
    message = [{"role":"user","content":main_request}]
    tools = [
    {
        "type":"function",
        "function":{
            "name":"get_current_weather",
            "description":"get the current weather in the given location",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"the name of the location"
                    },
                    "weather":{
                        "type":"string",
                        "description":"the current weather in the location"
                    }
                }
            },
            "required":["location"]
        }
    },
    {
        "type":"function",
        "function":{
            "name":"get_movie_recommendation",
            "description":"You are an assistant that recommends series based on their genre, rating, and year",
            "parameters":{
                "type":"object",
                "properties":{
                    "genre":{
                        "type":"string",
                        "description":"the genre of the movie"
                    },
                    "rating":{
                        "type":"number",
                        "description":"the rating of the movie"
                    },
                    "year":{
                        "type":"number",
                        "description":"the year of the movie"
                    },
                },
            },
            "required":["genre","rating","year"]
        }
    }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=message,
        tools=tools,
        tool_choice="auto"
    )
    message_response = response.choices[0].message
    print("first response: ",dict(message_response))

    tool_calls = message_response.tool_calls
    print("Tools",list(tool_calls))

    if tool_calls:
        available_functions = {
            "get_current_weather":get_current_weather,
            "get_movie_recommendation":get_movie_recommendation
        }
        message.append(message_response)
        for tool in tool_calls:
            function_name = tool.function.name
            function_args = json.loads(tool.function.arguments)
            function_to_call = available_functions[function_name]
            function_response = function_to_call(
                # location=function_args.get("location"),
                # weather = function_args.get("weather"),
                genre = function_args.get("genre"),
                rating = function_args.get("rating"),
                year = function_args.get("year"),
            )

            
            message.append({
                "tool_call_id":tool.id,
                "role":"tool",
                "name":function_name,
                "content":function_response
            })
        second_request = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=message
        )
        return second_request.choices[0].message.content
            

    



In [76]:
my_first_function_call("suggest me a comedy series with rating greater than 9 realesed in 2005")


first response:  {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [ChatCompletionMessageToolCall(id='call_17upHr5IOArTbIHWtvIMpzxS', function=Function(arguments='{"genre": "comedy", "rating": 9, "year": 2005}', name='get_movie_recommendation'), type='function'), ChatCompletionMessageToolCall(id='call_Yj2SZv9YGxPc1RMtLr4s2VW9', function=Function(arguments='{"genre": "comedy", "rating": 9, "year": 2005}', name='get_movie_recommendation'), type='function')]}
Tools [ChatCompletionMessageToolCall(id='call_17upHr5IOArTbIHWtvIMpzxS', function=Function(arguments='{"genre": "comedy", "rating": 9, "year": 2005}', name='get_movie_recommendation'), type='function'), ChatCompletionMessageToolCall(id='call_Yj2SZv9YGxPc1RMtLr4s2VW9', function=Function(arguments='{"genre": "comedy", "rating": 9, "year": 2005}', name='get_movie_recommendation'), type='function')]


'I\'m sorry, but I couldn\'t find a comedy movie with a rating greater than 9 released in 2005. However, I can recommend some highly-rated comedy movies from that year. Some of them are "Wedding Crashers" and "The 40-Year-Old Virgin." Both of these movies were released in 2005 and received positive reviews.'